© Jacob White 2025, All Rights Reserved

*Notes taken from Jackel's *Monte Carlo Methods in Finance*. All code is written by me.*

# Process-driven Sampling

## 3.1 Strong vs. Weak Convergence

- A time-discretised approximation $Y_\delta$ of steps not larger than $\delta$ is said to be of general **strong convergence** of order $\gamma$ if for any time horizon $T$ the approximation $Y_\delta(T)$ is guaranteed to converge to the exact solution $X(T)$ in expectation by absolute difference as $$E\left[|X(T) - Y_\delta(T)|\right] \leq c \delta^\gamma\tag{1}$$ for $\delta < \delta_0$ where $\delta_0$ is some positive constant, and for some constant $c$.

- **Weak convergence** of order $\beta$ requires only that the numerically calculated expectation of any function $g(\cdot)$ that is $2(\beta + 1)$-times continuously differentiable and of polynomial growth converges to the exact equivalent. I.e., for any $g \in C_P^{2(\beta + 1)}$, $$\left|E[g(X(T))] - E[g(Y_\delta(T))]\right| \leq c \delta^\beta \tag{2}$$ for all $\delta \leq \delta_0$ and some constant $c$.

- In general, any method of higher convergence order for the strong criterion is of superior performance for the purpose of option pricing.

- For many schemes, the convergence order may actually be exceeded for specific SDEs.

## 3.2 Numerical Solutions

For this section, we consider solutions to the general form of a stochastic differential equation: $$dX = a \ dt + b \ dW \tag{3}$$ where $a$ and $b$ can be functions of $X$ and $t$. In the multidimensional case of $m$ state variables $X_i$ driven by $d$ independent Wiener processes, we have $$dX_i = a_i(t, \boldsymbol X) \ dt + \sum_{j = 1}^d b_{ij}(t, \boldsymbol X) \ dW_j. \tag{4}$$

### The Euler Scheme

Denote the numerical approximation to the solution of (3) for a scheme over equal steps of size $\Delta t$ at time $n \cdot \Delta t$ as $Y(t_n)$. The Euler scheme is then given by $$Y(t_{n + 1}) = Y(t_n) + a(t_n, Y(t_n)) \Delta t + b(t_n, Y(t_n)) \Delta W. \tag{Euler}$$ 

- Euler scheme is of strong convergence order $\frac{1}{2}$.
- Workhorse of a numerical procedure can be used to test any other method.

**Example**: Let's implement the Euler scheme for a stochastic differential equation of the form $$dX = a \ dt + b \ dW$$ where $a$ and $b$ are constant.

Consulting the Euler scheme above, we have $$Y(t_{n + 1}) = Y(t_n) + a \Delta t + b \Delta W.$$ The Python implementation is below.

In [196]:
import numpy as np
from ipywidgets import interact, FloatSlider, IntSlider

# Euler scheme script to solve the stochastic differential equation 
# dX = a dt + b dW
# for a, b constant

def euler_scheme(a = 1.0, b = 0.5, n = 100):
    T: int = 1                                    # Time horizon
    seed: int = 42                                # Seed, and also the answer to the ultimate question of life, the universe, and everything
    np.random.seed(seed)                          # Set the random seed, comment out for the generation of random solutions
    dt = T/n                                      # Time increments 
    dW = np.random.normal(loc = 0, scale = np.sqrt(dt), size = n)        # Generate Wiener process increments
    time = np.linspace(0, T, n + 1)
    
    Y = [0]                                                              # Initial condition Y0 = 0
    for i in range(n):
        Y.append(Y[i] + a*dt + b*dW[i])                                  # Generates Euler scheme approximation to the SDE
    
    Y = np.array(Y)

    plt.plot(time, Y)
    plt.plot(time, a*time, linestyle = 'dashed', color = 'black', label = f'$y = {a} * t$')
    plt.legend(loc = "upper left")
    plt.title(f'Euler scheme approximation to the solution of the SDE $dX = {a}  dt + {b} dW$')
    plt.show()

interact(
    euler_scheme,
    a=FloatSlider(min=-5.0, max=5.0, step=0.1, value=1.0, description='a (drift)'),
    b=FloatSlider(min=0.0, max=2.0, step=0.1, value=0.5, description='b (volatility)'),
    n=IntSlider(min=10, max=1000, step=1, value=100, description='n (steps)')
)

interactive(children=(FloatSlider(value=1.0, description='a (drift)', max=5.0, min=-5.0), FloatSlider(value=0.…

<function __main__.euler_scheme(a=1.0, b=0.5, n=100)>

### The Milstein Scheme

The Milstein scheme involves the addition of the next order terms of the Itô-Taylor expansion of the SDE (3), which gives $$Y(t_{n + 1}) = Y(t_n) + a(t_n, Y(t_n)) \Delta t + b(t_n, Y(t_n)) \Delta W + \frac{1}{2} b \frac{\partial b(t, X)}{\partial X} \left[\Delta W^2 - \Delta t\right] \tag{Milstein}$$

Obviously, if $b$ is constant, we recover the original Euler scheme. Unlike the Euler scheme, its general multi-dimensional extension is not straightforward. It requires one to account for the Itô integrals involving the mixing terms $\sum_{j = 1}^d b_{ij} (t, \boldsymbol X) \ dW_j$. 